# 🚀 Quick Setup for Colab/Binder


## 📚 Example: What We'll Build

**Quick Preview**: This notebook will teach you to generate diverse text corpora using Large Language Models (LLMs) for psycholinguistic research.

**Example Output**: By the end of this session, you'll have generated text like this:

> *"The researcher examined the cognitive mechanisms underlying language comprehension, focusing on how semantic processing influences reading speed. Meanwhile, in another domain, the chef carefully balanced flavors in the experimental fusion dish, combining traditional techniques with innovative molecular gastronomy approaches."*

**What makes this valuable for research?**
- **Vocabulary Diversity**: Mix of technical (cognitive, semantic) and everyday (chef, flavors) words
- **Genre Variety**: Academic research + culinary arts in one corpus  
- **Natural Frequency Patterns**: Common words (the, in) appear frequently, specialized terms (gastronomy) appear rarely
- **Linguistic Complexity**: Varied sentence structures and lengths

**Research Goal**: Create word frequency predictors from LLM-generated text that can predict human reading times as accurately as traditional corpus-based measures.

---

🎯 **Learning Path**: Setup → Interactive Generation → Large-Scale Corpus → Frequency Analysis

In [1]:
# Environment Setup (Run this first on Colab/Binder)
import sys
import os

# Check if we're in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 Setting up Google Colab environment...")
    # Clone the repository if not already present
    if not os.path.exists('mlschool-text'):
        !git clone https://github.com/jobschepens/mlschool-text.git
        os.chdir('mlschool-text')
    else:
        os.chdir('mlschool-text')
    
    # Install requirements
    !pip install -q -r requirements_colab.txt
    print("✅ Colab setup complete!")

elif 'BINDER_LAUNCH_HOST' in os.environ:
    print("🔧 Binder environment detected - dependencies should already be installed")
    print("✅ Binder setup complete!")

else:
    print("💻 Local environment detected")
    print("Make sure you've run: pip install -r requirements.txt")

# Set working directory for consistent paths
if os.path.exists('mlschool-text') and not os.getcwd().endswith('mlschool-text'):
    os.chdir('mlschool-text')
elif os.getcwd().endswith('notebooks'):
    # If we're in the notebooks directory, go up one level
    os.chdir('..')

print(f"📁 Working directory: {os.getcwd()}")

# Verify key files are accessible
key_files = ['models.json', 'data/', 'scripts/', 'output/']
missing_files = []
for file_path in key_files:
    if not os.path.exists(file_path):
        missing_files.append(file_path)

if missing_files:
    print(f"⚠️ Warning: Cannot find {missing_files}")
    print("💡 Make sure you're in the correct directory")
else:
    print("✅ All key project files accessible")

print("🎯 Ready to start! You can now run the rest of the notebook.")

💻 Local environment detected
Make sure you've run: pip install -r requirements.txt
📁 Working directory: c:\GitHub\mlschool-text
✅ All key project files accessible
🎯 Ready to start! You can now run the rest of the notebook.


## 🔐 API Key Setup for Google Colab & Binder

**Important for cloud users**: Different platforms have different security considerations for API keys.

### For Google Colab Users (Recommended)
1. **Get an OpenRouter API key**: 
   - Go to [OpenRouter.ai](https://openrouter.ai) → Sign up → Get API key
   - Free tier available with generous limits!

2. **Store it securely in Colab**:
   - In Colab, click the **🔑 key icon** on the left sidebar (Secrets)
   - Click **"Add new secret"**
   - Name: `OPENROUTER_API_KEY`
   - Value: Paste your API key
   - Toggle **"Notebook access"** to ON
   - Click **"Save"**

### For Binder Users
Since Binder is ephemeral and public, you have three options:

1. **🔐 Enter API key manually** - Secure but temporary (key disappears when session ends)
2. **🎭 Demo mode** - Use realistic pre-generated examples (no API key needed)
3. **📊 Analysis-only mode** - Skip generation entirely, just analyze pre-existing data

### For Local Users
Create a `.env` file in the project root with:
```
OPENROUTER_API_KEY=your_key_here
```

### Step 2: Run the setup cell below

In [3]:
# Secure API Key Setup for Different Environments
import os
import getpass

# Check if we're in Colab
IN_COLAB = 'google.colab' in sys.modules
IN_BINDER = 'BINDER_LAUNCH_HOST' in os.environ

if IN_COLAB:
    # Use Colab's secure userdata feature
    try:
        from google.colab import userdata
        api_key = userdata.get('OPENROUTER_API_KEY')
        os.environ['OPENROUTER_API_KEY'] = api_key
        print("✅ API key loaded securely from Colab userdata!")
    except Exception as e:
        print("❌ Could not load API key from Colab userdata.")
        print("Make sure you've added 'OPENROUTER_API_KEY' to your Colab secrets!")
        print("Instructions: Click 🔑 in left sidebar → Add new secret → Name: OPENROUTER_API_KEY")
        
elif IN_BINDER:
    print("🚀 Binder environment detected!")
    print()
    print("📋 Options for API key in Binder:")
    print("1️⃣ Enter key manually (secure - not saved)")
    print("2️⃣ Use demo mode with pre-generated examples") 
    print("3️⃣ Skip generation and analyze existing data")
    print()
    
    choice = input("Choose option (1, 2, or 3): ").strip()
    
    if choice == "1":
        print("🔐 Enter your OpenRouter API key (input will be hidden):")
        api_key = getpass.getpass("API Key: ")
        if api_key.strip():
            os.environ['OPENROUTER_API_KEY'] = api_key.strip()
            print("✅ API key set temporarily for this session!")
            print("🔒 Key will be cleared when Binder session ends")
        else:
            print("❌ No API key entered")
    elif choice == "2":
        print("🎭 Demo mode selected - will use pre-generated examples")
        os.environ['DEMO_MODE'] = 'true'
        print("✅ Demo mode activated!")
    elif choice == "3":
        print("📊 Analysis mode selected - will skip generation")
        os.environ['ANALYSIS_ONLY'] = 'true'
        print("✅ Analysis-only mode activated!")
    else:
        print("❓ Invalid choice - defaulting to demo mode")
        os.environ['DEMO_MODE'] = 'true'
        
else:
    # Local environment - use .env file
    try:
        from dotenv import load_dotenv
        load_dotenv()
    except ImportError:
        pass  # dotenv not available, that's fine
    
    api_key = os.getenv("OPENROUTER_API_KEY")
    if api_key:
        print("✅ API key loaded from .env file!")
    else:
        print("❌ Please create a .env file with OPENROUTER_API_KEY=your_key_here")
        print("💡 Or set the environment variable directly")

# Verify the setup
if os.getenv("OPENROUTER_API_KEY"):
    print(f"🔑 API key configured (ends with: ...{os.getenv('OPENROUTER_API_KEY')[-4:]})")
elif os.getenv("DEMO_MODE"):
    print("🎭 Running in demo mode - will use example responses")
elif os.getenv("ANALYSIS_ONLY"):
    print("📊 Running in analysis-only mode - will skip text generation")
else:
    print("⚠️ No API key or mode selected - some features may not work")
    print("💡 Consider using demo mode to explore the concepts!")

✅ API key loaded from .env file!
🔑 API key configured (ends with: ...151c)


In [ ]:
# Path Utilities for Cross-Platform Compatibility
import os

def get_project_path(relative_path: str) -> str:
    """
    Get the correct path to project files regardless of working directory.
    
    Args:
        relative_path (str): Path relative to project root (e.g., 'models.json', 'data/file.csv')
    
    Returns:
        str: Absolute path to the file
    """
    # If we're already in the project root, use the path directly
    if os.path.exists(relative_path):
        return relative_path
    
    # If we're in notebooks/ subdirectory, go up one level
    parent_path = os.path.join('..', relative_path)
    if os.path.exists(parent_path):
        return parent_path
    
    # If we're in a cloned repo (like in Colab), try mlschool-text/ prefix
    colab_path = os.path.join('mlschool-text', relative_path)
    if os.path.exists(colab_path):
        return colab_path
    
    # Fallback to original path (will give clear error if file doesn't exist)
    return relative_path

def get_output_path(filename: str) -> str:
    """
    Get the correct path for output files (creates directory if needed).
    
    Args:
        filename (str): Output filename (e.g., 'corpus.txt', 'predictors.csv')
    
    Returns:
        str: Full path to output file
    """
    output_dir = get_project_path('output')
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    return os.path.join(output_dir, filename)

def check_file_access():
    """Check if key project files are accessible and show their paths."""
    key_files = {
        'models.json': 'Model configuration file',
        'data/': 'Data directory', 
        'scripts/': 'Scripts directory',
        'output/': 'Output directory'
    }
    
    print("📁 File accessibility check:")
    all_good = True
    
    for file_path, description in key_files.items():
        full_path = get_project_path(file_path)
        if os.path.exists(full_path):
            print(f"   ✅ {description}: {full_path}")
        else:
            print(f"   ❌ {description}: {full_path} (not found)")
            all_good = False
    
    return all_good

# Run the check
if check_file_access():
    print("🎯 All project files accessible!")
    print(f"💾 Output files will be saved to: {get_project_path('output')}")
else:
    print("⚠️ Some files are missing - check your working directory")
    print(f"Current directory: {os.getcwd()}")
    print("💡 Try re-running the environment setup cell above")

📁 File accessibility check:
   ✅ Model configuration file: models.json
   ✅ Data directory: data/
   ✅ Scripts directory: scripts/
   ✅ Output directory: output/
🎯 All project files accessible!
💾 Output files will be saved to: output


# Part 1: Generating a Diverse LLM Corpus for Frequency Analysis
## Session 1: From LLM Generation to Word Frequency

**Learning Objectives:**
- Generate a diverse text corpus using an LLM to mirror the vocabulary found in large-scale datasets like the English Crowdsourcing Project (ECP).
- Understand how prompt engineering across different genres (news, technical, fiction) can create a representative word frequency list.
- Calculate word frequencies from the generated text to create a custom `llm_frequency` predictor.
- Export the frequency data for comparative analysis in the next session.

**Session Structure:**
- **Setup & API Configuration** (10 minutes)
- **Diverse Corpus Generation** (25 minutes)
- **Frequency Calculation & Export** (10 minutes)

---

💡 **Research Context:** Our goal is to create a high-quality `llm_frequency` predictor. The ECP dataset, which we use for validation in Notebook 2, contains a wide vocabulary from many sources (general use, dictionaries, etc.). To create a comparable predictor, we must generate a corpus that is equally diverse. A simple corpus (e.g., only children's stories) is insufficient. This session focuses on generating a varied corpus to capture a broad slice of the English language.

## 1.1 Setup and API Configuration

Let's set up our environment and configure the API client to generate our corpus.


In [4]:
# Environment Setup and API Configuration
import os
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import time
from typing import Dict, List, Any

print("🚀 LLM Text Generation Session")
print("=" * 35)
print("Setting up environment for corpus generation...")

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ Base environment configured")

🚀 LLM Text Generation Session
Setting up environment for corpus generation...
✅ Base environment configured



Let's set up our environment and configure the API client to generate our corpus.

First, we need a way to communicate with a Large Language Model (LLM). Instead of a complex setup, we'll use a single function that calls the **OpenRouter API**. OpenRouter allows us to access many different models (like GPT, Claude, Llama, etc.) through one consistent interface.

**Key Steps:**
1. **API Key**: If you followed the setup instructions above, your API key should already be configured
2. **Function Definition**: We define `call_openrouter`, which handles the API request  
3. **Testing**: We'll run a quick test to ensure everything is configured correctly

*If the test fails, please check the API key setup instructions in the cells above.*

In [5]:
# Enhanced API Function with Demo Mode Support
import os
import requests
import json
import random

def call_openrouter(prompt: str, model_name: str = "openai/gpt-3.5-turbo") -> str:
    """
    Calls the OpenRouter API with a specified model and prompt, or returns demo responses.
    
    Args:
        prompt (str): The user prompt to send to the LLM.
        model_name (str): The model to use (e.g., "openai/gpt-3.5-turbo").
        
    Returns:
        str: The generated text content from the LLM or demo response.
    """
    
    # Check for demo mode
    if os.getenv("DEMO_MODE"):
        return get_demo_response(prompt)
    
    # Check for analysis-only mode
    if os.getenv("ANALYSIS_ONLY"):
        return "Analysis-only mode: Text generation skipped. Using pre-existing data for analysis."
    
    # Regular API call
    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        error_msg = """
        ❌ OPENROUTER_API_KEY not found!
        
        📋 Setup Instructions:
        🔹 Colab: Add key to Secrets (🔑 icon) → Run setup cell above
        🔹 Local: Create .env file with OPENROUTER_API_KEY=your_key_here
        🔹 Binder: Re-run setup cell and choose option 1, 2, or 3
        """
        raise ValueError(error_msg)
        
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json',
        "HTTP-Referer": "http://localhost:3000",
        "X-Title": "ML-School-Psycholinguistics"
    }
    
    data = {
        'model': model_name,
        'messages': [{'role': 'user', 'content': prompt}],
        'max_tokens': 400,
        'temperature': 0.7
    }
    
    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=data,
            timeout=60
        )
        response.raise_for_status()
        result = response.json()
        
        if 'choices' in result and len(result['choices']) > 0:
            return result['choices'][0]['message']['content'].strip()
        else:
            return f"Error: Unexpected response format - {result}"
            
    except requests.exceptions.RequestException as e:
        print(f"❌ API request failed: {e}")
        return f"Error: {e}"
    except KeyError as e:
        print(f"❌ Response format error: {e}")
        return f"Error: Unexpected response format"

def get_demo_response(prompt: str) -> str:
    """
    Returns realistic demo responses for different types of prompts.
    """
    demo_responses = {
        "psycholinguistics": [
            "Psycholinguistics is the study of how language is processed in the brain, combining insights from psychology, linguistics, and neuroscience.",
            "The field of psycholinguistics examines how humans acquire, comprehend, and produce language through cognitive processes.",
            "Researchers in psycholinguistics investigate the mental mechanisms underlying language comprehension and production."
        ],
        "science": [
            "The process of photosynthesis allows plants to convert sunlight into chemical energy, producing oxygen as a byproduct that sustains most life on Earth.",
            "Gravity is a fundamental force that shapes the structure of the universe, from holding planets in orbit to influencing the formation of galaxies.",
            "DNA contains the genetic instructions that guide the development and functioning of all living organisms."
        ],
        "story": [
            "Sarah discovered an old journal in her grandmother's attic, filled with pressed flowers and mysterious entries about a hidden garden that seemed to change with the seasons.",
            "The lighthouse keeper noticed something unusual in the fog that night - a ship that appeared to be sailing backwards through time.",
            "When Maya opened the peculiar music box, she found herself transported to a world where colors had sounds and melodies had taste."
        ],
        "technical": [
            "To optimize database performance, consider implementing proper indexing strategies, query optimization techniques, and regular maintenance procedures.",
            "Machine learning algorithms can be broadly categorized into supervised, unsupervised, and reinforcement learning approaches, each suitable for different types of problems.",
            "The software development lifecycle includes planning, analysis, design, implementation, testing, deployment, and maintenance phases."
        ],
        "general": [
            "Learning a new skill requires patience, practice, and persistence, but the rewards of mastering something challenging are immense.",
            "Climate change represents one of the most significant challenges of our time, requiring coordinated global action and innovative solutions.",
            "The rapid advancement of technology continues to transform how we work, communicate, and understand the world around us."
        ]
    }
    
    # Choose appropriate demo responses based on prompt content
    prompt_lower = prompt.lower()
    if "psycholinguist" in prompt_lower or "language" in prompt_lower:
        responses = demo_responses["psycholinguistics"]
    elif "story" in prompt_lower or "character" in prompt_lower or "creative" in prompt_lower:
        responses = demo_responses["story"]
    elif "technical" in prompt_lower or "how to" in prompt_lower or "algorithm" in prompt_lower:
        responses = demo_responses["technical"]
    elif "science" in prompt_lower or "explain" in prompt_lower:
        responses = demo_responses["science"]
    else:
        responses = demo_responses["general"]
    
    return f"[DEMO] {random.choice(responses)}"

# --- Test the API function ---
print("🚀 Testing OpenRouter API function...")
try:
    test_prompt = "Write a single sentence about psycholinguistics."
    test_response = call_openrouter(test_prompt)
    print(f"✅ API Test Successful!")
    print(f"   Prompt: {test_prompt}")
    print(f"   Response: {test_response}")
    print(f"   Response length: {len(test_response)} characters")
    
    # Show mode information
    if os.getenv("DEMO_MODE"):
        print("   Mode: 🎭 Demo mode active")
    elif os.getenv("ANALYSIS_ONLY"):
        print("   Mode: 📊 Analysis-only mode active")
    else:
        print("   Mode: 🌐 Live API mode active")
        
except ValueError as e:
    print(f"❌ {e}")
except Exception as e:
    print(f"❌ An unexpected error occurred during the test: {e}")
    print("💡 Try running the API key setup cell above first!")

🚀 Testing OpenRouter API function...
✅ API Test Successful!
   Prompt: Write a single sentence about psycholinguistics.
   Response: Psycholinguistics is the study of how language is processed and understood by the human brain.
   Response length: 94 characters
   Mode: 🌐 Live API mode active


---

## ✅ Setup Complete!

If the test above was successful, you're ready to proceed! Your environment now has:
- ✅ All required libraries installed
- ✅ API key configured securely  
- ✅ OpenRouter connection verified
- ✅ Ready for corpus generation

**Next:** Continue to the interactive text generation section below to start exploring different prompts and models.

## 1.2 Interactive Text Generation 
Now for the hands-on part. We will generate short text samples using different models and prompts to understand how these choices influence the resulting text. This is the core of corpus generation methodology.

**Our goals here are to:**
- **Explore Model Differences**: See how different models (e.g., a creative one vs. a technical one) respond to the same prompt.
- **Understand Prompt Engineering**: Learn how small changes to a prompt can alter the style, vocabulary, and complexity of the output.
- **Observe Corpus Characteristics**: Get a feel for the kind of text each strategy produces.

We'll start by defining a few models available through OpenRouter.

In [9]:
# --- Model and Prompt Experimentation ---
import json

# Define the specific models we want to test in this notebook.
# We will filter the larger list from models.json using these IDs.
selected_model_ids = {
    # "qwen/qwen3-30b-a3b-instruct-2507",
    "meta-llama/llama-3.3-8b-instruct:free", # Note: Using the free tier ID from the JSON
    "deepseek/deepseek-chat-v3.1:free"      # Note: Using the free tier ID from the JSON
}

# Load models from the JSON file using our path utility
try:
    models_path = get_project_path('models.json')
    with open(models_path, 'r') as f:
        models_data = json.load(f)['models']
    
    # Create a full dictionary of available models, preferring free tiers
    available_models = {}
    for model in models_data:
        if model.get('type') != 'image_generation':
            model_id = model.get('free_tier', {}).get('id', model['id'])
            available_models[model['name']] = model_id
            
    # Filter the available models to get only the ones we selected
    models_to_test = {
        name: model_id for name, model_id in available_models.items()
        if model_id in selected_model_ids
    }
    
    print(f"✅ Successfully loaded and selected {len(models_to_test)} models from 'models.json'.")
    if len(models_to_test) < len(selected_model_ids):
        print("   ⚠️ Some selected models were not found in 'models.json'.")

except (FileNotFoundError, json.JSONDecodeError, KeyError) as e:
    print(f"⚠️ Could not load models from 'models.json': {e}")
    print("   Falling back to default models.")
    # Fallback to a default set if the file is missing or invalid
    models_to_test = {
        "OpenAI GPT-3.5": "openai/gpt-3.5-turbo",
        "Llama3 8B": "meta-llama/llama-3-8b-instruct"
    }

# Define a simple prompt
prompt = "Write a short, imaginative story about a librarian who discovers a book that writes itself."

print(f"\n🎯 Selected models for testing:")
for name, model_id in models_to_test.items():
    print(f"   • {name}: {model_id}")

✅ Successfully loaded and selected 2 models from 'models.json'.

🎯 Selected models for testing:
   • Meta Llama 3.3 8B Instruct: meta-llama/llama-3.3-8b-instruct:free
   • DeepSeek-V3.1: deepseek/deepseek-chat-v3.1:free


In [ ]:
# Practical Example: Using the OpenRouter Call Function
print("🎯 PRACTICAL EXAMPLE: OpenRouter Function in Action")
print("=" * 55)

# Example 1: Generate text for different genres using the same model
print("\n📚 Example 1: Multi-Genre Text Generation")
print("-" * 40)

# Define different prompts for various genres
example_prompts = {
    "Academic": "Explain the concept of neuroplasticity in simple terms for students.",
    "Creative": "Write a short paragraph about a mysterious door that appears in different places.",
    "Technical": "Describe the basic steps for setting up a secure password policy.",
    "News": "Write a brief news report about a new archaeological discovery.",
    "Conversational": "Give advice to someone learning to cook their first meal."
}

# Select a model to use (use the first available model)
if models_to_test:
    selected_model_name, selected_model_id = list(models_to_test.items())[0]
    print(f"🤖 Using model: {selected_model_name} ({selected_model_id})")
    
    # Generate text for each genre
    generated_texts = {}
    for genre, prompt in example_prompts.items():
        print(f"\n📝 {genre} Genre:")
        print(f"   Prompt: {prompt}")
        
        try:
            response = call_openrouter(prompt, selected_model_id)
            generated_texts[genre] = response
            
            # Display first 100 characters of response
            preview = response[:100] + "..." if len(response) > 100 else response
            print(f"   Response: {preview}")
            print(f"   Length: {len(response)} characters")
            
        except Exception as e:
            print(f"   ❌ Error: {e}")
            generated_texts[genre] = f"Error: {e}"
    
    # Analysis of generated texts
    print(f"\n📊 Quick Analysis:")
    total_words = 0
    unique_words = set()
    
    for genre, text in generated_texts.items():
        if not text.startswith("Error:"):
            # Simple word extraction
            words = re.findall(r'\b\w+\b', text.lower())
            total_words += len(words)
            unique_words.update(words)
            
            print(f"   {genre}: {len(words)} words, {len(set(words))} unique")
    
    print(f"\n🎯 Summary:")
    print(f"   Total words generated: {total_words}")
    print(f"   Total unique words: {len(unique_words)}")
    print(f"   Vocabulary diversity: {len(unique_words)/total_words:.3f}" if total_words > 0 else "   No valid text generated")

else:
    print("❌ No models available for testing")

# Example 2: Compare different models with the same prompt
print(f"\n\n🔄 Example 2: Model Comparison")
print("-" * 40)

if len(models_to_test) >= 2:
    comparison_prompt = "Describe the process of learning a new language in one paragraph."
    print(f"🎯 Prompt: {comparison_prompt}")
    
    for model_name, model_id in models_to_test.items():
        print(f"\n🤖 {model_name}:")
        try:
            response = call_openrouter(comparison_prompt, model_id)
            
            # Basic analysis
            words = re.findall(r'\b\w+\b', response.lower())
            sentences = response.count('.') + response.count('!') + response.count('?')
            
            print(f"   Response: {response}")
            print(f"   📊 Stats: {len(words)} words, ~{sentences} sentences")
            
        except Exception as e:
            print(f"   ❌ Error: {e}")
            
elif len(models_to_test) == 1:
    print("Only one model available - skipping comparison")
else:
    print("No models available for comparison")

# Example 3: Demonstrate temperature and creativity control
print(f"\n\n🌡️ Example 3: Understanding Model Parameters")
print("-" * 45)

creativity_prompt = "Write a creative opening line for a science fiction story."

if models_to_test:
    model_name, model_id = list(models_to_test.items())[0]
    
    print(f"🎭 Testing creativity with: {model_name}")
    print(f"📝 Prompt: {creativity_prompt}")
    
    # Note: The temperature is set to 0.7 in our function
    # In a real implementation, you might want to make this configurable
    for i in range(3):
        print(f"\n   Attempt {i+1}:")
        try:
            response = call_openrouter(creativity_prompt, model_id)
            print(f"   → {response}")
        except Exception as e:
            print(f"   ❌ Error: {e}")

print(f"\n✅ OpenRouter function examples complete!")
print(f"💡 These examples show how to generate diverse text for corpus building.")
print(f"🚀 Ready to scale up for large corpus generation!")

### Analysis and Exploration

Look at the outputs from the different models. Consider the following questions:
- **Vocabulary**: Which model used more common words? Which used more unusual or complex words?
- **Style**: Did one model produce a more creative or narrative style? Was another more direct or factual?
- **Length & Structure**: Were there noticeable differences in sentence length or structure?

This hands-on exploration is the first step in understanding how to build a corpus that is well-suited for psycholinguistic analysis. A good corpus needs to be diverse enough to capture the wide range of words people encounter in their daily lives.

## 1.3 Large-Scale Corpus Generation (via Script)

While we can generate small amounts of text interactively in this notebook, creating a multi-million word corpus requires a dedicated script that can run for a long time, save its progress, and handle potential interruptions.

For this purpose, we will use `scripts/script-1-gen.py`.

**How it Works:**
1.  **Configuration File**: The script's behavior is controlled by a `.json` configuration file. We have created `scripts/config_2m_llama.json` specifically for generating our 2-million-word Llama 3.3 corpus.
2.  **Seed vs. No Seed Words**: The script can use general, self-contained prompts across different genres to generate a diverse corpus without being biased by a specific word list or it can use seed words to "inspire" the LLM to write into a certain direction.
3.  **State Management**: It continuously saves its progress to a state file (`.json`), so if the script is stopped, it can be resumed later without losing work or money.

**To run the generation process, you would execute the following command in your terminal:**
```bash
python scripts/script-1-gen.py --config scripts/config_2m_llama.json
```

This process will run in the background to generate the `large_corpus_2m_llama.txt` file in the `output/` directory, which is essential for the next stages of our research. Note that you can change certain settings by making a new config file. For the purpose of this summer school session, a corpus has been pre-generated for you.

## Session 1 Summary & Next Steps

**What We've Accomplished:**
- **Simplified API Access**: We set up a clean, reusable function to connect to various LLMs via OpenRouter.
- **Interactive Exploration**: We experimented with different models and prompts to see how they affect text generation.
- **Systematic Corpus Generation**: We created a small but diverse corpus by generating text from prompts across multiple genres.

We will now move on to **Notebook 2**, where we will:
- Load our custom-generated frequency predictor.
- Compare it against established, human-derived frequency norms (like SUBTLEX).
- Validate its predictive power against real human reading time data.

---

**Next**: Open `notebook2_corpus_analysis.ipynb` to begin the analysis phase.